In [1]:
import os
import pandas as pd

In [2]:
df = pd.read_csv("../data_given/winequality.csv")

In [3]:
overview = df.describe()

In [7]:
overview.loc[["min","max"]].to_json("schema_in.json")

In [9]:
class NotInRange(Exception):
    def __init__(self, message="value not in range"):
#         self.input_ = input_
        self.message = message
        super().__init__(self.message)

In [10]:
raise NotInRange

NotInRange: value not in range